In [1]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from itertools import product
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load data
X_train_input = np.load("Data/X_train.npy")  # shape (700, 6)
Y_train_input = np.load("Data/Y_train.npy")  # shape (700,)

# Random train/test split (100 for test, 600 for training)
X_train_cv, X_test_holdout, Y_train_cv, Y_test_holdout = train_test_split(
    X_train_input, Y_train_input, test_size=300, random_state=42, shuffle=True
)

# Scale features
scaler = StandardScaler()
X_train_cv_scaled = scaler.fit_transform(X_train_cv)
X_test_holdout_scaled = scaler.transform(X_test_holdout)

In [3]:
scaler = StandardScaler()
X_train_cv_scaled = scaler.fit_transform(X_train_cv)
X_test_holdout_scaled = scaler.transform(X_test_holdout)

In [4]:
# RBF transformation function
def rbf_transform(X, centers, sigma):
    diff = X[:, np.newaxis, :] - centers[np.newaxis, :, :]
    return np.exp(-np.sum(diff**2, axis=2) / (2 * sigma**2))

# Hyperparameter grids - Initial
#sigma_list = [0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0]
#n_centers_list = [20, 30, 40, 50, 100, 125, 150, 200]
#alpha_list = [0.001, 0.005, 0.01, 0.1, 1.0, 10.0]

# Hyperparameter grids - Specialized
sigma_list = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0]
n_centers_list = [150, 175, 200, 225, 250]
alpha_list = [0.0005, 0.00075, 0.001, 0.005, 0.01]


# K-Fold CV
kf = KFold(n_splits=5, shuffle=True, random_state=42)

best_r2 = -np.inf
best_params = None

# Example: results = [{'sigma': s, 'n_centers': n, 'alpha': a, 'mean_r2': r2}, ...]
results = []

In [5]:
for sigma, n_centers, alpha in product(sigma_list, n_centers_list, alpha_list):
    
    r2_scores = []
    
    for train_idx, val_idx in kf.split(X_train_cv_scaled):
        X_tr, X_val = X_train_cv_scaled[train_idx], X_train_cv_scaled[val_idx]
        Y_tr, Y_val = Y_train_cv[train_idx], Y_train_cv[val_idx]
        
        # KMeans centers on training fold
        kmeans = KMeans(n_clusters=n_centers, random_state=42).fit(X_tr)
        centers = kmeans.cluster_centers_
        
        # Transform both training and validation data
        X_tr_rbf = rbf_transform(X_tr, centers, sigma)
        X_val_rbf = rbf_transform(X_val, centers, sigma)
        
        # Train Ridge regression
        model = Ridge(alpha=alpha)
        model.fit(X_tr_rbf, Y_tr)
        
        # Predict on validation fold
        Y_val_pred = model.predict(X_val_rbf)
        r2_scores.append(r2_score(Y_val, Y_val_pred))
    
    mean_r2 = np.mean(r2_scores)
    results.append({'sigma': sigma, 'n_centers': n_centers, 'alpha': alpha, 'mean_r2': mean_r2})
    
    if mean_r2 > best_r2:
        best_r2 = mean_r2
        best_params = {'sigma': sigma, 'n_centers': n_centers, 'alpha': alpha}

print("Best Cross Validation R2:", best_r2)
print("Best Hyperparameters:", best_params)

Best Cross Validation R2: 0.9606201088393866
Best Hyperparameters: {'sigma': 2.5, 'n_centers': 250, 'alpha': 0.0005}


In [6]:
# Refit model on all 600 training samples using best hyperparameters
sigma_best = best_params['sigma']
n_centers_best = best_params['n_centers']
alpha_best = best_params['alpha']

# KMeans centers on full training fold
kmeans = KMeans(n_clusters=n_centers_best, random_state=42).fit(X_train_cv_scaled)
centers_best = kmeans.cluster_centers_

# Transform training and test data
X_train_rbf = rbf_transform(X_train_cv_scaled, centers_best, sigma_best)
X_test_rbf = rbf_transform(X_test_holdout_scaled, centers_best, sigma_best)

# Train final model
final_model = Ridge(alpha=alpha_best)
final_model.fit(X_train_rbf, Y_train_cv)

# Predict on held-out 100 samples
Y_test_pred = final_model.predict(X_test_rbf)
test_r2 = r2_score(Y_test_holdout, Y_test_pred)
print("R2 on held-out 100 test samples:", test_r2)

R2 on held-out 100 test samples: 0.9697717973342667


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

# Valores preditos
Y_pred = final_model.predict(X_test_rbf)
test_r2 = r2_score(Y_test_holdout, Y_pred)
print("R² on held-out test samples:", test_r2)

# Scatter plot: y_real vs y_pred
plt.figure(figsize=(6,6))
plt.scatter(Y_test_holdout, Y_pred, alpha=0.7)
plt.plot([Y_test_holdout.min(), Y_test_holdout.max()],
         [Y_test_holdout.min(), Y_test_holdout.max()],
         'r--', label='Ideal fit')
plt.xlabel("Valores reais")
plt.ylabel("Valores previstos")
plt.title("RBF Ridge: Predição vs Real")
plt.legend()
plt.grid(True)
plt.show()
